---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [65]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan","Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State","RegionName"]  )'''
    import pandas as pd
    import numpy as np
    import re
    
    univ_df = pd.DataFrame(columns=['State', 'RegionName'])
    line_count = 0
    for line in open("university_towns.txt"):  # read and process one line at a time
        if line.count('[edit]'):
            univ_df.loc[line_count]= [line.split('[')[0].strip(), np.NaN]
        else:
            univ_df.loc[line_count] = [np.NaN, re.split('\(',line)[0].strip()]
        line_count += 1
    return univ_df  
#     with open("university_towns.txt") as univ_file:
#         univ_df['Line'] = univ_file.readlines()
    
#     univ_df['State'] = univ_df['Line'].apply(lambda x: x.split('[')[0].strip() if x.count('[edit]') else np.NaN)
#     univ_df['RegionName'] = univ_df['Line'].apply(lambda x: re.split('\(',x)[0].strip() if not x.count('[edit]') else np.NaN)
   
       
    univ_df['State'].fillna(method='ffill', inplace=True)
    univ_df = univ_df[-pd.isnull(univ_df['RegionName'])]
    univ_df.reset_index(inplace=True)
    
    
    
    return univ_df[['State', 'RegionName']]
get_list_of_university_towns()

,State,RegionName
0,Alabama,NaN
1,NaN,Auburn
2,NaN,Florence
3,NaN,Jacksonville
4,NaN,Livingston
5,NaN,Montevallo
6,NaN,Troy
7,NaN,Tuscaloosa
8,NaN,Tuskegee
9,Alaska,NaN


In [4]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    import pandas as pd
    import numpy as np
    gdp = pd.read_excel("gdplev.xls", skiprows=4, parse_cols='E:G')
    gdp.rename(columns={'Unnamed: 0':'quarter', 'Unnamed: 1':'gdp', 'Unnamed: 2':'gdp_norm'}, inplace=True)
    gdp = gdp[-gdp['quarter'].isnull()]
    gdp = gdp[gdp['quarter'] >= '2000q1']
    gdp.reset_index(drop=True, inplace=True)
    gdp['gdp_growth'] = gdp['gdp_norm'].diff()
    
    recession_start = [False for i in range(0, len(gdp['gdp_growth']))]
    for i in range(0, len(gdp['gdp_growth'])):
        if ((i > 1 ) & (i < len(gdp['gdp_growth']) - 1)) and \
            ((gdp['gdp_growth'][i-1] > 0) & (gdp['gdp_growth'][i] < 0) & (gdp['gdp_growth'][i + 1] < 0)):
            recession_start[i] = True
        else:
            recession_start[i] = False
    gdp = gdp[recession_start]
    
    gdp.reset_index(drop=True, inplace = True)
    return gdp['quarter'].iloc[0]
    
get_recession_start()

'2008q3'

In [5]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    import pandas as pd
    import numpy as np
    gdp = pd.read_excel("gdplev.xls", skiprows=4, parse_cols='E:G')
    gdp.rename(columns={'Unnamed: 0':'quarter', 'Unnamed: 1':'gdp', 'Unnamed: 2':'gdp_norm'}, inplace=True)
    gdp = gdp[-gdp['quarter'].isnull()]
    gdp = gdp[gdp['quarter'] >= '2000q1']
    gdp.reset_index(drop=True, inplace=True)
    gdp['gdp_growth'] = gdp.gdp_norm.diff()
    recession_start = [False for i in range(0, len(gdp['gdp_growth']))]
    
    for i in range(0, len(gdp['gdp_growth'])):
        if ((i > 1 ) & (i < len(gdp['gdp_growth']) - 1)) and \
            ((gdp['gdp_growth'][i-1] > 0) & (gdp['gdp_growth'][i] < 0) & (gdp['gdp_growth'][i + 1] < 0)):
            recession_start[i] = True
            start_pos = i
        else:
            recession_start[i] = False
   
    recession_end =  [False for i in range(0, len(gdp['gdp_growth']))]
            
    for i in range(start_pos, len(gdp['gdp_growth']) - 1):
        if (gdp['gdp_growth'][i] > 0) & (gdp['gdp_growth'][i - 1] > 0):
            recession_end[i] = True
            break
       
    return gdp[recession_end]['quarter'].iloc[0]
    
get_recession_end()

'2009q4'

In [6]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    import pandas as pd
    import numpy as np
    gdp = pd.read_excel("gdplev.xls", skiprows=4, parse_cols='E:G')
    gdp.rename(columns={'Unnamed: 0':'quarter', 'Unnamed: 1':'gdp', 'Unnamed: 2':'gdp_norm'}, inplace=True)
    gdp = gdp[-gdp.quarter.isnull()]
    gdp = gdp[gdp.quarter >= '2000q1']
    gdp.reset_index(drop=True, inplace=True)
    gdp['growth'] = gdp.gdp_norm.diff()
    gdp['growth_next'] = gdp.growth.shift(-1)
    gdp['growth_prev'] = gdp.growth.shift(1)
    
    def recession_start (row):
        if (row['growth'] < 0) & (row['growth_next'] < 0) & (row['growth_prev'] > 0):
            row['recession_start'] = True
            row['recession'] = True
        else:
            row['recession_start'] = False
            row['recession'] = False
        return row
    gdp = gdp.apply(recession_start, axis=1)
    
    gdp.ix[gdp.index > gdp[gdp.recession_start].index[0], 'recession_start'] = True
    gdp.ix[gdp.index > gdp[gdp.recession].index[0], 'recession'] = True
    def recession_end (row):
        if (row['recession_start'] & (row['growth'] > 0) & (row['growth_prev'] > 0)):
            row['recession_end'] = True
        else:
            row['recession_end'] = False
        return row
    gdp = gdp.apply(recession_end, axis=1)
    gdp.ix[gdp.index > gdp[gdp.recession_end].index[0], 'recession'] = False
     
    gdp = gdp[gdp.recession == True]
    return gdp[gdp.gdp_norm == gdp.gdp_norm.min()]['quarter'].iloc[0]
get_recession_bottom()

'2009q2'

In [27]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    import pandas as pd
    import numpy as np
    
    # housing = pd.read_csv("City_Zhvi_AllHomes.csv", usecols=["State", "RegionName", "1996-05", "1996-07"])
    housing = pd.read_csv("City_Zhvi_AllHomes.csv")
    # Use this dictionary to map state names to two letter acronyms
    states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}
    #states = {y:x for x, y in states.items()}
    housing['State'].replace(states, inplace=True)
    housing['RegionName'] = housing['RegionName'].apply(lambda x : x.strip())
    housing = housing.set_index(['State', 'RegionName'])
    housing1 = housing.ix[:,49:] 
    housing2 = pd.DataFrame()
    columns = housing1.columns.values
    
    for i in range(0, len(columns) -1, 3):
        month = columns[i].split('-')
        quarter = month[0] + ('q1' if int(month[1]) <=3 else 'q2' if int(month[1]) <= 6 \
                             else 'q3' if int(month[1]) <= 9 else 'q4')
        housing2[quarter] = housing1.iloc[:,i:i+3].mean(axis=1)
    housing2.sort_index(inplace=True)
    # partial label housing2['Alabama'] is not working as expected by multiindex document
    # has to use housing2.loc['Alabama']
    return housing2


In [48]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    import pandas as pd
    from scipy import stats
        
    # Create a data frame of mean price ratio. price_ratio=quarter_before_recession/recession_bottom
    housing_price = convert_housing_data_to_quarters()
    housing_price['price_ratio'] = housing_price.apply(lambda x : x['2008q2']/x['2009q2'], axis=1)
    price_ratio = housing_price[['price_ratio']]
    price_ratio.reset_index(inplace = True)
    college_town = get_list_of_university_towns()
    college_town['Colledge'] = True
    price_ratio = pd.merge(price_ratio, college_town, how="left")
    #return merged[merged['Colledge'].notnull()]
    # remove rows where there is no price ratio
    price_ratio = price_ratio[-price_ratio.price_ratio.isnull()]
    price_ratio.reset_index(drop= True, inplace = True)
    
    non_colledge = price_ratio[price_ratio.Colledge.isnull()]
    colledge = price_ratio[price_ratio.Colledge.notnull()]
    test_result = stats.ttest_ind(non_colledge['price_ratio'], colledge['price_ratio'])
    colledge_mean = float(colledge.mean())
    noncolledge_mean = float(non_colledge.mean())
    
    if (colledge_mean < noncolledge_mean):
        better = colledge_mean
    else:
        better = noncolledge_mean
        
    different = True if test_result[1] < 0.01 else False
    return (different, test_result[1], better)
    
run_ttest()

(True, 0.002724063704761164, 1.0545645283851426)